In [1]:
from ae_models import ae_model
from keras.models import Model
import keras
from keras.layers import Input,Dense,Dropout
from keras import backend as K
import numpy as np
import pandas as pd
import random

Using TensorFlow backend.


In [2]:
class_num=10
trainx=pd.read_csv(f'../data/CRWU/x_1750_{class_num}_f.csv').iloc[:4000]
trainy=pd.read_csv(f'../data/CRWU/y_1750_{class_num}.csv').iloc[:4000]
#x_test=pd.read_csv('../data/JN/x_600_2_f.csv').iloc[:3000].values
#y_test=pd.read_csv('../data/JN/y_600_2.csv').iloc[:3000].values
x_test=pd.read_csv(f'../data/CRWU/x_1772_{class_num}_f.csv').iloc[:4000].values
y_test=pd.read_csv(f'../data/CRWU/y_1772_{class_num}.csv').iloc[:4000].values
index =list(trainx.index)
random.Random(0).shuffle(index)
num=int(len(index)/5*5)
x_train,y_train=trainx.iloc[index[:num],:].values,trainy.iloc[index[:num],:].values
#x_test,y_test=trainx.iloc[index[num:],:].values,trainy.iloc[index[num:],:].values
#标准化处理
def trans(data):
    nm=np.zeros(data.shape)
    for i in range(len(data)):
        nm[i,:]=(data[i,:])/np.max(data[i,:])
    return nm
x_train=trans(x_train)
x_test=trans(x_test)
data_X=np.vstack([x_train,x_test])

### 自编码

In [3]:
## contractive auto-encoder
lam=10e-5
def contractive_loss(y_pred, y_true):
    mse = K.mean(K.square(y_true - y_pred), axis=1)
    contractive=0
    for ename in ['eh1','eh2']:
        W = K.variable(value=ae_mode.get_layer(ename).get_weights()[0])  # N x N_hidden
        W = K.transpose(W)  # N_hidden x N
        h = ae_mode.get_layer(ename).output
        dh = h * (1 - h)  # N_batch x N_hidden
        contractive+= lam * K.sum(dh**2 * K.sum(W**2, axis=1), axis=1)
    return mse + contractive
#定义损失函数2（默认为categorical_crossentropy）
def contractive_loss2(y_pred, y_true):
    cost = K.categorical_crossentropy(y_true,y_pred)
    contractive=0
    for ename in ['eh0','eh1','LR']:
        W = K.variable(value=encoder_mode.get_layer(ename).get_weights()[0])  # N x N_hidden
        W = K.transpose(W)  # N_hidden x N
        h = encoder_mode.get_layer(ename).output
        dh = h * (1 - h)  # N_batch x N_hidden
        contractive+= lam * K.sum(dh**2 * K.sum(W**2, axis=1), axis=1)
    return cost + contractive

## non-negetive-constrain
l_non=1e-3
def non_con(y_pred, y_true):
    mse = K.mean(K.square(y_true - y_pred), axis=1)
    non_constrain=0
    for ename in ['eh0','eh1']:
        weights = K.variable(value=ae_mode.get_layer(ename).get_weights()[0]) # N x N_hidden
        non_constrain+= (K.sum(K.abs(weights))-K.sum(K.abs(weights)))**2
    return mse + non_constrain*l_non
#定义损失函数2（默认为categorical_crossentropy）
def non_con2(y_pred, y_true):
    cost = K.categorical_crossentropy(y_true,y_pred)
    non_constrain=0
    for ename in ['eh0','eh1','LR']:
        W = K.variable(value=encoder_mode.get_layer(ename).get_weights()[0])  # N x N_hidden
        non_constrain+= (K.sum(K.abs(weights))-K.sum(K.abs(weights)))**2
    return cost + non_constrain*l_non

### MMD
def compute_kernel(x, y):
    x_size = K.shape(x)[0]
    y_size = K.shape(y)[0]
    dim = K.shape(x)[1]
    tiled_x = K.tile(K.reshape(x, [x_size, 1, dim]), [1, y_size, 1])
    tiled_y = K.tile(K.reshape(y, [1, y_size, dim]), [x_size, 1, 1])
    return K.exp(-K.mean(K.square(tiled_x - tiled_y), axis=2) / K.cast(dim, 'float32'))

def compute_mmd(x, y):
    x_kernel = compute_kernel(x, x)
    y_kernel = compute_kernel(y, y)
    xy_kernel = compute_kernel(x, y)
    return K.mean(x_kernel) + K.mean(y_kernel) - 2 * K.mean(xy_kernel)

def custom_loss(encode_t,encode_s,ae_out,train_xr):

    loss_mmd = compute_mmd(encode_t,encode_s)
    'Then, also get the reconstructed loss'
    loss_nll = K.mean(K.square(train_xr - ae_out))
    loss = loss_nll + loss_mmd*0.1
    return loss
def custom_loss2(encode_s,encode_t,LR,y_true):
    loss_mmd = compute_mmd(encode_t,encode_s)
    'Then, also get the reconstructed loss'
    loss_cost = K.categorical_crossentropy(y_true,LR)
    loss = loss_cost + loss_mmd*0.1
    return loss

In [4]:
K.clear_session()

#第一个input
input_dim=1024
input_s=Input(shape=(input_dim,))
input_t=Input(shape=(input_dim,))

ae=ae_model(input_dim,[400,200,64],0.1,0.01,0)
ae_out=ae.auto_encode(input_s)
encode_t=ae.encode(input_t)

#分类
encode_s=ae.encode(input_s)
LR=Dense(class_num,activation='softmax',name='LR')(encode_s)

#编译自编码
ae_mode=Model(inputs=[input_s,input_t],outputs=ae_out)
loss = custom_loss(encode_s,encode_t,ae_out,input_s)
ae_mode.add_loss(loss)
ae_mode.compile(optimizer='adam')
hist=ae_mode.fit([x_train,x_test],epochs=10,batch_size=128,shuffle=True)

D:\anaconda\lib\site-packages\keras\engine\training_utils.py:819: UserWarning: Output dense_4 missing from loss dictionary. We assume this was done on purpose. The fit and evaluate APIs will not be expecting any data to be passed to dense_4.
  'be expecting any data to be passed to {0}.'.format(name))



Epoch 1/10
4000/4000 [==============================] - 3s 682us/step - loss: 5.3488
Epoch 2/10
4000/4000 [==============================] - 1s 130us/step - loss: 1.2658
Epoch 3/10
4000/4000 [==============================] - 1s 130us/step - loss: 0.3268
Epoch 4/10
4000/4000 [==============================] - 1s 133us/step - loss: 0.1054
Epoch 5/10
4000/4000 [==============================] - 1s 129us/step - loss: 0.0403
Epoch 6/10
4000/4000 [==============================] - 1s 141us/step - loss: 0.0190
Epoch 7/10
4000/4000 [==============================] - 1s 129us/step - loss: 0.0117
Epoch 8/10
4000/4000 [==============================] - 1s 136us/step - loss: 0.0091
Epoch 9/10
4000/4000 [==============================] - 1s 132us/step - loss: 0.0085
Epoch 10/10
4000/4000 [==============================] - 1s 130us/step - loss: 0.0080


In [5]:
y_true=Input(shape=(class_num,))
encoder_mode=Model(inputs=[input_s,input_t,y_true],outputs=LR)
loss = custom_loss2(encode_s,encode_t,LR,y_true)
encoder_mode.add_loss(loss)
encoder_mode.compile(optimizer='adam',metrics=['categorical_accuracy'])
encoder_mode.fit([x_train,x_test,y_train],epochs=50,batch_size=128,shuffle=True)

D:\anaconda\lib\site-packages\keras\engine\training_utils.py:819: UserWarning: Output LR missing from loss dictionary. We assume this was done on purpose. The fit and evaluate APIs will not be expecting any data to be passed to LR.
  'be expecting any data to be passed to {0}.'.format(name))


Epoch 1/50
4000/4000 [==============================] - 1s 339us/step - loss: 7.0457
Epoch 2/50
4000/4000 [==============================] - 1s 142us/step - loss: 2.4730
Epoch 3/50
4000/4000 [==============================] - 1s 145us/step - loss: 1.2328
Epoch 4/50
4000/4000 [==============================] - 1s 153us/step - loss: 0.8059
Epoch 5/50
4000/4000 [==============================] - 1s 150us/step - loss: 0.6075
Epoch 6/50
4000/4000 [==============================] - 1s 147us/step - loss: 0.5074
Epoch 7/50
4000/4000 [==============================] - 1s 145us/step - loss: 0.4462
Epoch 8/50
4000/4000 [==============================] - 1s 150us/step - loss: 0.3992
Epoch 9/50
4000/4000 [==============================] - 1s 147us/step - loss: 0.3698
Epoch 10/50
4000/4000 [==============================] - 1s 145us/step - loss: 0.3474
Epoch 11/50
4000/4000 [==============================] - 1s 145us/step - loss: 0.3309
Epoch 12/50
4000/4000 [==============================] - 1s 144

In [6]:
pred=Model(inputs=input_s,outputs=LR)
c=pred.predict(x_test)
np.mean(np.where(c.argmax(axis=1)==y_test.argmax(axis=1),1,0))

In [7]:
def compute_mmd2(Z):
    x=Z[0]
    y=Z[1]
    x_kernel = compute_kernel(x, x)
    y_kernel = compute_kernel(y, y)
    xy_kernel = compute_kernel(x, y)
    return K.mean(x_kernel) + K.mean(y_kernel) - 2 * K.mean(xy_kernel)

In [8]:
#计算mmd
from keras.layers import Lambda
inp=Input(shape=(input_dim,))
encoder_=ae.encode(inp)
emode=Model(inp,encoder_)

input_s=Input(shape=(input_dim,))
input_t=Input(shape=(input_dim,))
encode1=emode(input_s)
encode2=emode(input_t)
out_mmd=Lambda(compute_mmd2)([encode1,encode2])
mmd_cost=Model(inputs=[input_s,input_t],outputs=out_mmd)

In [20]:
input_s=encoder_data()
input_t=encoder_data()
inp1=input_s.input
inp2=input_t.input
for layer in input_s.layers:
    layer.name =layer.name + str("_2")

out_mmd=Lambda(compute_mmd2)([input_s.output,input_t.output])
mmd_cost=Model(inputs=[inp1,inp2],outputs=out_mmd)

In [11]:
mmd_cost.predict([x_test[:50],x_train[:50]])

array([3.0994415e-06, 3.0994415e-06, 3.0994415e-06, 3.0994415e-06,
       3.0994415e-06, 3.0994415e-06, 3.0994415e-06, 3.0994415e-06,
       3.0994415e-06, 3.0994415e-06, 3.0994415e-06, 3.0994415e-06,
       3.0994415e-06, 3.0994415e-06, 3.0994415e-06, 3.0994415e-06,
       3.0994415e-06, 3.0994415e-06, 3.0994415e-06, 3.0994415e-06,
       3.0994415e-06, 3.0994415e-06, 3.0994415e-06, 3.0994415e-06,
       3.0994415e-06, 3.0994415e-06, 3.0994415e-06, 3.0994415e-06,
       3.0994415e-06, 3.0994415e-06, 3.0994415e-06, 3.0994415e-06,
       5.7220459e-06, 5.7220459e-06, 5.7220459e-06, 5.7220459e-06,
       5.7220459e-06, 5.7220459e-06, 5.7220459e-06, 5.7220459e-06,
       5.7220459e-06, 5.7220459e-06, 5.7220459e-06, 5.7220459e-06,
       5.7220459e-06, 5.7220459e-06, 5.7220459e-06, 5.7220459e-06,
       5.7220459e-06, 5.7220459e-06], dtype=float32)

score=encoder_mode.evaluate([x_train,x_test,y_train])
print(score)